# Overview

This baseline has a test score of `0.144`.

This is a baseline approach for testing. I used `"Category: " + df["ConstructName"] + ". Question: " + df["QuestionText"] + " Given Answer: " + Answer[A-D]Text` are vectorized using a unmodified `baai-bge-large-en` model, the top 25 high cosine similarity misconception ID are submitted as inference results.

# Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

# Data Load

In [2]:
train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")

misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
sample_submission = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv")

# Preprocess

In [42]:
def make_all_question_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_question_text"] = "Category: " + df["ConstructName"] + ". Question: " + df["QuestionText"]
    return df

# train = make_all_question_text(train)
test = make_all_question_text(test)

In [43]:
test

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,all_question_text
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Category: Use the order of operations to carry...
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Category: Simplify an algebraic fraction by fa...
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Category: Calculate the range from a list of d...


In [44]:
# test["all_question_text"][0]
test["ConstructName"][0]

'Use the order of operations to carry out calculations involving powers'

In [45]:
def wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.melt(
        df[
            [
                "QuestionId",
                "all_question_text",
                "CorrectAnswer",
                "AnswerAText",
                "AnswerBText",
                "AnswerCText",
                "AnswerDText"
            ]
        ],
        id_vars=["QuestionId", "all_question_text", "CorrectAnswer"],
        var_name='Answer',
        value_name='value'
    )

    return df

# train_long = wide_to_long(train)
test_long = wide_to_long(test)
test_long
# test_long["all_question_text"][0]

,QuestionId,all_question_text,CorrectAnswer,Answer,value
0,1869,Category: Use the order of operations to carry...,A,AnswerAText,\( 3 \times(2+4)-5 \)
1,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerAText,\( m+1 \)
2,1871,Category: Calculate the range from a list of d...,B,AnswerAText,Only\nTom
3,1869,Category: Use the order of operations to carry...,A,AnswerBText,\( 3 \times 2+(4-5) \)
4,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerBText,\( m+2 \)
5,1871,Category: Calculate the range from a list of d...,B,AnswerBText,Only\nKatie
6,1869,Category: Use the order of operations to carry...,A,AnswerCText,\( 3 \times(2+4-5) \)
7,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerCText,\( m-1 \)
8,1871,Category: Calculate the range from a list of d...,B,AnswerCText,Both Tom and Katie
9,1869,Category: Use the order of operations to carry...,A,AnswerDText,Does not need brackets


In [48]:
def make_all_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_text"] = df["all_question_text"] + " Given Answer: " + df["value"]
    return df

# train_long = make_all_text(train_long)
test_long = make_all_text(test_long)
test_long

,QuestionId,all_question_text,CorrectAnswer,Answer,value,all_text
0,1869,Category: Use the order of operations to carry...,A,AnswerAText,\( 3 \times(2+4)-5 \),Category: Use the order of operations to carry...
1,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerAText,\( m+1 \),Category: Simplify an algebraic fraction by fa...
2,1871,Category: Calculate the range from a list of d...,B,AnswerAText,Only\nTom,Category: Calculate the range from a list of d...
3,1869,Category: Use the order of operations to carry...,A,AnswerBText,\( 3 \times 2+(4-5) \),Category: Use the order of operations to carry...
4,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerBText,\( m+2 \),Category: Simplify an algebraic fraction by fa...
5,1871,Category: Calculate the range from a list of d...,B,AnswerBText,Only\nKatie,Category: Calculate the range from a list of d...
6,1869,Category: Use the order of operations to carry...,A,AnswerCText,\( 3 \times(2+4-5) \),Category: Use the order of operations to carry...
7,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerCText,\( m-1 \),Category: Simplify an algebraic fraction by fa...
8,1871,Category: Calculate the range from a list of d...,B,AnswerCText,Both Tom and Katie,Category: Calculate the range from a list of d...
9,1869,Category: Use the order of operations to carry...,A,AnswerDText,Does not need brackets,Category: Use the order of operations to carry...


In [49]:
test_long.all_text.values[0]


'Category: Use the order of operations to carry out calculations involving powers. Question: \\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ? Given Answer: \\( 3 \\times(2+4)-5 \\)'

In [50]:
# sort
# train_long = train_long.sort_values(["QuestionId", "Answer"]).reset_index(drop=True)
test_long = test_long.sort_values(["QuestionId", "Answer"]).reset_index(drop=True)
test_long

,QuestionId,all_question_text,CorrectAnswer,Answer,value,all_text
0,1869,Category: Use the order of operations to carry...,A,AnswerAText,\( 3 \times(2+4)-5 \),Category: Use the order of operations to carry...
1,1869,Category: Use the order of operations to carry...,A,AnswerBText,\( 3 \times 2+(4-5) \),Category: Use the order of operations to carry...
2,1869,Category: Use the order of operations to carry...,A,AnswerCText,\( 3 \times(2+4-5) \),Category: Use the order of operations to carry...
3,1869,Category: Use the order of operations to carry...,A,AnswerDText,Does not need brackets,Category: Use the order of operations to carry...
4,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerAText,\( m+1 \),Category: Simplify an algebraic fraction by fa...
5,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerBText,\( m+2 \),Category: Simplify an algebraic fraction by fa...
6,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerCText,\( m-1 \),Category: Simplify an algebraic fraction by fa...
7,1870,Category: Simplify an algebraic fraction by fa...,D,AnswerDText,Does not simplify,Category: Simplify an algebraic fraction by fa...
8,1871,Category: Calculate the range from a list of d...,B,AnswerAText,Only\nTom,Category: Calculate the range from a list of d...
9,1871,Category: Calculate the range from a list of d...,B,AnswerBText,Only\nKatie,Category: Calculate the range from a list of d...


# 计算标签MisconceptionName的emb

In [52]:
# 数据准备

labels = misconception_mapping['MisconceptionName'].values


In [53]:
# 加载模型
from transformers import AutoTokenizer, AutoModel
import torch
# Sentences we want sentence embeddings for

device = "cuda:0"

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/baai-bge-large-en')
model = AutoModel.from_pretrained('/kaggle/input/baai-bge-large-en')
model.eval()
model.to(device)
print("finish")

finish


In [54]:
from tqdm import tqdm
MisconceptionName = list(misconception_mapping['MisconceptionName'].values)
per_gpu_batch_size = 8


def prepare_inputs(text, tokenizer, device):
    tokenizer_outputs = tokenizer.batch_encode_plus(
        text,
        padding=True,
        return_tensors='pt',
        max_length=520,
        truncation=True)
    result = {
        'input_ids': tokenizer_outputs.input_ids.to(device),
        'attention_mask': tokenizer_outputs.attention_mask.to(device),
    }
    return result


all_ctx_vector = []
for mini_batch in tqdm(
        range(0, len(MisconceptionName[:]), per_gpu_batch_size)):
    mini_context = MisconceptionName[mini_batch:mini_batch
                                           + per_gpu_batch_size]
    encoded_input = prepare_inputs(mini_context,tokenizer,device)
    sentence_embeddings = model(**encoded_input)[0][:, 0]

    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    
    all_ctx_vector.append(sentence_embeddings.detach().cpu().numpy())

all_ctx_vector = np.concatenate(all_ctx_vector, axis=0)
# all_ctx_vector = np.array(all_ctx_vector).astype('float32')
# faiss_index = faiss.IndexFlatIP(all_ctx_vector.shape[-1])
# faiss_index.add(all_ctx_vector)
# sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
print("Sentence embeddings:", sentence_embeddings.shape)

100%|██████████| 324/324 [00:14<00:00, 22.72it/s]

Sentence embeddings: torch.Size([3, 1024])


In [55]:
test_texts = list(test_long.all_text.values)
all_text_vector = []
per_gpu_batch_size = 8

for mini_batch in tqdm(
        range(0, len(test_texts[:]), per_gpu_batch_size)):
    mini_context = test_texts[mini_batch:mini_batch
                                           + per_gpu_batch_size]
    encoded_input = prepare_inputs(mini_context,tokenizer,device)
    sentence_embeddings = model(
        **encoded_input)[0][:, 0]
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    
    all_text_vector.append(sentence_embeddings.detach().cpu().numpy())

all_text_vector = np.concatenate(all_text_vector, axis=0)
print(all_text_vector.shape)

100%|██████████| 2/2 [00:00<00:00,  6.78it/s]

(12, 1024)


# Predict

In [56]:
test_cos_sim_arr = cosine_similarity(all_text_vector, all_ctx_vector)
test_sorted_indices = np.argsort(-test_cos_sim_arr, axis=1)

In [57]:
test_sorted_indices[:, :25]

array([[2488, 1316, 1672, 1119,   15, 2586,  373, 1941, 2202,   74, 1999,
        2306,  969, 2518,  158, 1432, 2532, 1666, 2221,  752,  466, 1971,
        1814, 1576,  256],
       [2488, 1316, 1672,   15, 1119,  373, 2586, 2202, 1941,   74, 2306,
        1999,  158,  969, 2518, 2532, 2221, 1432, 1666,  752,  466, 1971,
         315, 1814, 1576],
       [2488, 1316, 1672,   15, 1119,  373, 2586, 1941, 2202,   74, 2306,
        1999,  969,  158, 2532, 2518, 1432, 1666, 2221,  752,  466, 1971,
        2131,  315, 1576],
       [2488,  373, 1316,  969,   74,  315, 1672,   15, 2586, 1666, 2532,
        1432, 1119, 2202,  158, 2306, 1941,  871, 1999, 2131,  580, 1576,
         466, 2518,  752],
       [2398, 2307,  979, 2066, 1540,  848, 1548,  885,   80, 1593,  825,
        2068, 2142, 1469, 2078,  606, 1909,  519,  659,   59, 1804, 2444,
         363, 1825,  546],
       [2398, 2307,  979, 2066, 1540,  848, 1548,  885,   80, 1593,  825,
        2142, 2068, 1469, 2078,  606,  659,  519, 1

# Make Submit File

In [58]:
test_long["Answer_alphabet"] = test_long["Answer"].str.extract(r'Answer([A-Z])Text$')
test_long["QuestionId_Answer"] = test_long["QuestionId"].astype("str") + "_" + test_long["Answer_alphabet"]
test_long["MisconceptionId"] = test_sorted_indices[:, :25].tolist()
test_long["MisconceptionId"] = test_long["MisconceptionId"].apply(lambda x: ' '.join(map(str, x)))
# filter correct row
test_long = test_long[test_long["CorrectAnswer"] != test_long["Answer_alphabet"]]
submission = test_long[["QuestionId_Answer", "MisconceptionId"]].reset_index(drop=True)

In [59]:
submission.head(10)

,QuestionId_Answer,MisconceptionId
0,1869_B,2488 1316 1672 15 1119 373 2586 2202 1941 74 2...
1,1869_C,2488 1316 1672 15 1119 373 2586 1941 2202 74 2...
2,1869_D,2488 373 1316 969 74 315 1672 15 2586 1666 253...
3,1870_A,2398 2307 979 2066 1540 848 1548 885 80 1593 8...
4,1870_B,2398 2307 979 2066 1540 848 1548 885 80 1593 8...
5,1870_C,2398 2307 2066 1540 979 1548 848 885 80 1593 8...
6,1871_A,1183 1382 1015 1302 238 312 1287 644 392 372 5...
7,1871_C,1382 1183 1015 1302 312 644 238 1287 372 392 2...
8,1871_D,1183 1382 312 1287 644 1302 1015 591 238 392 2...


In [60]:
sample_submission.head(10)

,QuestionId_Answer,MisconceptionId
0,1869_A,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,1869_B,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
2,1869_C,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
3,1870_B,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
4,1870_C,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
5,1870_D,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
6,1871_A,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
7,1871_C,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
8,1871_D,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...


In [61]:
submission.to_csv("submission.csv", index=False)